#Instructions to run lfp_analysis.py

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
import lfp_analysis
import os
import glob

###Convert the video files to mp4 format by passing the experiment directory

Experiment directory should be the root directory that contains all experiment files.

If needed, loop through the function below to convert the video files from .h264 to mp4 format.

In [ ]:
experiment_dir = "/Volumes/chaitra/reward_competition_extension/data"
lfp_analysis.convert_to_mp4(experiment_dir)

# Loop through all the directories in the experiment directory
for directory in os.listdir(experiment_dir):
    if os.path.isdir(os.path.join(experiment_dir, directory)):
        lfp_analysis.convert_to_mp4(os.path.join(experiment_dir, directory))

###Input parameters to create LfpExperiment object

In [ ]:
experiment_dir = "/Volumes/chaitra/reward_competition_extension/data"
experiment_description = "chaitra_test"

###Input parameters to create LfpRecordingObject

channel_map_path: Path to the channel mapping excel file

events_path (video): Path to the events excel file

labels_path (good): Path to the labels excel file

input_dir: Path to the directory containing the \*.rec files

sleap_path: Path to the directory containing the sleap files

phy_path: Path to the directory containing the phy curation files. Should be in the format of phy_curation/experiment_session/phy/*


condition
competition closeness

In [ ]:
channel_map_path = "input/channel_mapping.xlsx"
events_path = "input/events.xlsx"
labels_path = "input/labels.xlsx"

#This list has to be in the same order as the files in the input directory
list_subjects = ["1.4_and_1.2", "2.1_and_2.2", "1.1_and_1.2", "2.1_and_2.4"]
#TODO: dictionary with key as .rec and values as subject

input_dir = "/Volumes/chaitra/reward_competition_extension/data/standard/2023_06_*/*.rec"
sleap_path = "/Volumes/chaitra/reward_competition_extension/data/proc/sleap/"
output_path = "/Volumes/chaitra/reward_competition_extension/data/outputs/"

if not os.path.exists(output_path):
    os.makedirs(output_path)

#tagging label excel values
encoding_dict = {'Subj 1 blocking Subj 2': "competitive",
    'Subj 2 Only': "no_comp",
    'Subj 2 blocking Subj 1': "competitive",
    'Subj 1 then Subj 2': "competitive",
    'Subj 1 Only': "no_comp",
    'Subj 2 then Subj 1': "competitive",
    'Close Call': "competitive",
    'After trial': "no_comp"}

In [ ]:
if not os.path.exists(channel_map_path) or not os.path.exists(events_path) or not os.path.exists(labels_path):
    raise Exception("Please provide the correct path for channel_map_path, events_path and labels_path")

if not os.path.exists(sleap_path):
    raise Exception("Please provide the correct path for sleap_path")

#Check if input directory exists, wildcard is allowed
if not glob.glob(input_dir):
    raise Exception("Please provide the correct path for input_dir")

if len(list_subjects) == 0:
    raise Exception("Provided list_subjects is empty")

#print input dir files
num_rec = 0
for file in glob.glob(input_dir):
    if file.endswith(".rec"):
        num_rec += 1
        print(file + "\n")

if len(list_subjects) != num_rec:
    raise Exception("Number of subjects in list_subjects is not equal to the number of files in the input directory")

In [ ]:
# Create Expr object
my_expr = lfp_analysis.LfpExperiment(experiment=experiment_description)

In [ ]:
counter = 0
# Loop through input directory and create LFP objects for each session
for file in glob.glob(input_dir):
    lfp = lfp_analysis.LfpRecordingObject(path=file,
                                           channel_map_path=channel_map_path,
                                           sleap_path=sleap_path,
                                           events_path=events_path,
                                           labels_path=labels_path,
                                           encoding_dict=encoding_dict,
                                           experiment_name=experiment_description, output_path=output_path,
                                           subject=list_subjects[counter])
    my_expr.add_lfp_object(lfp_object=lfp, subject=lfp.subject)
    counter += 1